In [1]:
# Packages importation
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [2]:
# Data loading
cars_path = "../group_00095_shared_workspace/car_models_cluster_v1.csv"
car_models = pd.read_csv(cars_path, sep=";", index_col=[0])
car_models.head(5)

,Brand,year,Average Urban Consumption,Average extra-urban consumption,Average mixed consumption,Average of CO2 (g per km),Average CO type I,clust
Ford Fiesta Sedan 2012,Ford,2012,4.900000,3.600,4.100000,107.000000,0.421,1
FIAT 500 Convertible 2012,FIAT,2012,5.725000,4.175,4.775000,111.000000,0.318,1
Chevrolet Corvette ZR1 2012,Chevrolet,2012,19.000000,8.900,12.600000,293.000000,0.508,3
Lamborghini Gallardo LP 570-4 Superleggera 2012,Lamborghini,2012,21.406667,9.860,14.086667,337.133333,0.000,3
Honda Accord Coupe 2012,Honda,2012,6.900000,4.500,5.400000,141.000000,0.215,1


In [3]:
car_models.groupby('clust').describe()['Average of CO2 (g per km)']

,count,mean,std,min,25%,50%,75%,max
clust,,,,,,,,
1,53.0,166.510491,42.788813,101.000000,136.538462,153.533569,201.717391,281.000000
2,34.0,198.726111,46.703281,106.000000,164.195652,190.663158,232.719907,293.000000
3,13.0,346.561050,52.001497,286.909091,296.333333,339.250000,358.307692,441.721312


In [4]:
car_models['clust'].value_counts()

1    53
2    34
3    13
Name: clust, dtype: int64

In [5]:
!pip install pytorch-lightning

In [43]:
import torch
from PIL import Image, ImageFile
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import VisionDataset
import pytorch_lightning as pl
from torchvision.models import efficientnet_b0
import torchvision.transforms as T
import os
import pandas as pd
ImageFile.LOAD_TRUNCATED_IMAGES = True

loss_f = torch.nn.CrossEntropyLoss()

class Classifier(pl.LightningModule):
    def __init__(self, lr=1e-3):
        super().__init__()
        self.backbone = efficientnet_b0(pretrained=True)
        self.backbone.classifier[1] = torch.nn.Linear(in_features=1280, out_features=3, bias=True)
        self.lr = 2e-4

    def forward(self, x):
        # use forward for inference/predictions
        embedding = self.backbone(x)
        return embedding

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = loss_f(y_hat, F.one_hot(y.squeeze()-1,3).float())
        self.log("train_loss", loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = loss_f(y_hat, F.one_hot(y.squeeze()-1,3).float())
        self.log("valid_loss", loss, on_step=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = loss_f(y_hat, F.one_hot(y.squeeze()-1,3).float())
        self.log("test_loss", loss)

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        x, y = batch
        return self(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr,)

In [44]:
class CarData(VisionDataset):
    def __init__(self):
        super().__init__("../datasets/datasets_train/train")
        self.transform = T.Compose(
            [
                T.Resize(256),
                T.RandomCrop(224),
                T.ColorJitter(brightness=.5, hue=.3),
                T.RandomHorizontalFlip(p=0.5),
                T.PILToTensor(),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ]
        )
        self.co2 = pd.read_csv('../group_00095_shared_workspace/car_models_cluster_v1.csv',sep=";", index_col=0, squeeze=True)["clust"].to_dict()
        self.cars = pd.read_csv('../datasets/datasets_train/train_annotation/_annotation.csv', index_col=1, squeeze=True)["models"].to_dict()
        self.images = []
        for img in os.listdir(self.root):
            if 'checkpoint' not in img and self.cars[img] != ' ':
                self.images.append(img)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        with open(os.path.join(self.root,self.images[idx]), "rb") as f:
            img = Image.open(f).convert("RGB")
        return self.transform(img),torch.Tensor([self.co2[self.cars[self.images[idx]]]]).to(torch.int64)

In [45]:
dataset = CarData()
train_size = 4*len(dataset)//5
train, val = random_split(dataset, [train_size,len(dataset)-train_size])
train_loader = DataLoader(train, batch_size=16,num_workers=55, pin_memory=True,shuffle=True)
val_loader = DataLoader(val, batch_size=52,num_workers=55, pin_memory=True)
model = Classifier()
# Initialize a trainer
trainer = pl.Trainer(
    gpus=1,
    max_epochs=100,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [46]:
trainer.fit(model,train_loader,val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type         | Params
------------------------------------------
0 | backbone | EfficientNet | 4.0 M 
------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.046    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
